# Úloha zjištění sentimentu z textu

# Dataset IMDB

Jedná se o datovou sadu 25 000 recenzí filmů z IMDB, označených podle sentimentu (pozitivní/negativní). 

Recenze byly předem zpracovány a každá recenze je zakódována jako seznam slovních indexů (celých čísel). 

Pro větší pohodlí jsou slova indexována podle celkové četnosti v souboru dat, takže například celé číslo "3" kóduje třetí nejčastější slovo v datech. 

To umožňuje rychlé operace filtrování, jako např: "zohlednit pouze 10 000 nejčastějších slov, ale vyřadit 20 nejčastějších slov".

Podle konvence "0" neoznačuje konkrétní slovo, ale používá se pro zakódování tokenu položky.

In [1]:
%matplotlib inline

In [2]:
from keras.datasets import imdb
import matplotlib.pyplot as plt
import numpy as np

# 1. Načtení dat

In [3]:
vocabulary_size = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocabulary_size)

Vstupní data - slova z indexu

In [ ]:
print (x_train[0])

In [ ]:
# Stáhnutí indexu slov
word_idx = imdb.get_word_index()
 
# Původně indexové číslo hodnoty není klíč.
# Je tedy nutný převod indexu jako klíče a slov jako hodnot 
word_idx = {i: word for word, i in word_idx.items()}
 
# zobrazení recenze
print([word_idx[i] for i in x_train[0]])

In [ ]:
# První rezence má 218 slov
len(x_train[0])

In [ ]:
# Vypsání minimální a maximální délky recenzí
print("Maximální délka recenze: ", len(max((x_train+x_test), key=len)))
print("Minimální délka recenze: ", len(min((x_train+x_test), key=len)))

In [ ]:
# kategorie rezencí - positive (1), negative (0)
print(np.unique(y_train))

# 2. Příprava dat

In [9]:
# Knihovna tensonrflow má funkce pro práci se sekvencemi
from tensorflow.keras.preprocessing import sequence

# Z každé recenze vezmem prvních 400 slov. Pokud recenze není dostatečně dlouhá, doplníme ji prázdným slovem, respektive číslem 0
max_words = 400
 
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)
 
x_valid, y_valid = x_train[:64], y_train[:64]
x_train_, y_train_ = x_train[64:], y_train[64:]

In [ ]:
# Délka první upravené rezence
print (len(x_train[0]))

In [ ]:
# zobrazení první recenze
x_train[0]

# 3. Jednoduchý RNN model

In [13]:
from keras.layers import SimpleRNN, Dense, Embedding
from keras.models import Sequential

In [14]:
# stanovení velikosti vloženého slova na 32.
embd_len = 32

In [15]:
RNN_model = Sequential(name="Simple_RNN")
RNN_model.add(Embedding(vocabulary_size,
                        embd_len,
                        input_length=max_words))
 
RNN_model.add(SimpleRNN(128,
                        activation='tanh',
                        return_sequences=False))
RNN_model.add(Dense(1, activation='sigmoid'))

In [ ]:
RNN_model.summary()

In [17]:
# Jedná se o klasifikační model se dvěma třídami
# proto používáme ztrátovou funkci binary_crossentropy
RNN_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
rnn_history = RNN_model.fit(x_train_, y_train_,
                        batch_size=64,
                        epochs=5,
                        verbose=1,
                        validation_data=(x_valid, y_valid))

In [ ]:
RNN_model.save('rnn_simple')

In [ ]:
RNN_model.evaluate(x_test, y_test)

In [ ]:
# historie učení
fig1 = plt.figure()
plt.plot(rnn_history.history['loss'], label='Train Loss')
plt.plot(rnn_history.history['accuracy'], label='Train Accuracy')
plt.legend(loc="right")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Počet epoch')
plt.show()   

# 4. GRU model

In [25]:
from keras.layers import GRU
gru_model = Sequential(name="GRU_Model")
gru_model.add(Embedding(vocabulary_size,
                        embd_len,
                        input_length=max_words))
gru_model.add(GRU(128,
                  activation='tanh',
                  return_sequences=False))
gru_model.add(Dense(1, activation='sigmoid'))

In [ ]:
gru_model.summary()

In [27]:
gru_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
gru_history = gru_model.fit(x_train_, y_train_,
                         batch_size=64,
                         epochs=5,
                         verbose=1,
                         validation_data=(x_valid, y_valid))

In [ ]:
gru_model.save('rnn_gru')

In [ ]:
gru_model.evaluate(x_test, y_test)

In [ ]:
# historie učení
fig2 = plt.figure()                
plt.plot(gru_history.history['loss'], label='Train Loss')
plt.plot(gru_history.history['accuracy'], label='Train Accuracy')
plt.plot(gru_history.history['val_loss'], label='Validation Loss')
plt.plot(gru_history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc="right")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Počet epoch')
plt.show()   

# 5. LTSM

In [32]:
from keras.layers import LSTM

In [33]:
lstm_model = Sequential(name="LSTM_Model")
lstm_model.add(Embedding(vocabulary_size,
                         embd_len,
                         input_length=max_words))
lstm_model.add(LSTM(128,
                    activation='relu',
                    return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))

In [ ]:
lstm_model.summary()

In [35]:
lstm_model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
ltsm_history = lstm_model.fit(x_train_, y_train_,
                          batch_size=64,
                          epochs=5,
                          verbose=1,
                          validation_data=(x_valid, y_valid))

In [ ]:
lstm_model.save('rnn_ltsm')

In [ ]:
lstm_model.evaluate(x_test, y_test)

In [ ]:
# historie učení
fig3 = plt.figure()                
plt.plot(ltsm_history.history['loss'], label='Train Loss')
plt.plot(ltsm_history.history['accuracy'], label='Train Accuracy')
plt.plot(ltsm_history.history['val_loss'], label='Validation Loss')
plt.plot(ltsm_history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc="right")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Počet epoch')
plt.show() 

# 6. Bi-directional LSTM Model

In [40]:
from keras.layers import Bidirectional

In [41]:
bi_lstm_model = Sequential(name="Bidirectional_LSTM")
bi_lstm_model.add(Embedding(vocabulary_size,
                            embd_len,
                            input_length=max_words))
bi_lstm_model.add(Bidirectional(LSTM(128,
                                     activation='tanh',
                                     return_sequences=False)))
bi_lstm_model.add(Dense(1, activation='sigmoid'))

In [ ]:
bi_lstm_model.summary()

In [42]:
bi_lstm_model.compile(
  loss="binary_crossentropy",
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
bi_lstm_history = bi_lstm_model.fit(x_train_, y_train_,
                             batch_size=64,
                             epochs=5,
                             validation_data=(x_test, y_test))

In [ ]:
bi_lstm_model.save('rnn_bi_ltsm')

In [ ]:
bi_lstm_model.evaluate(x_test, y_test)

In [ ]:
# historie učení
fig4 = plt.figure()                
plt.plot(bi_lstm_history.history['loss'], label='Train Loss')
plt.plot(bi_lstm_history.history['accuracy'], label='Train Accuracy')
plt.plot(bi_lstm_history.history['val_loss'], label='Validation Loss')
plt.plot(bi_lstm_history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc="right")
plt.title('Loss, accuracy')
plt.ylabel('Loss, accuracy')
plt.xlabel('Počet epoch')
plt.show() 